In [6]:
!pip install -q google-generativeai pdfplumber


In [9]:
import google.generativeai as genai
import pdfplumber
from google.colab import files
from IPython.display import display, Markdown
import getpass

# 🔐 Get Gemini API key
GEMINI_API_KEY = getpass.getpass("🔑 Enter your Gemini API Key: ")
genai.configure(api_key=GEMINI_API_KEY)

# 📥 Upload PO and SO PDF files
print("📥 Upload Purchase Order (PO) PDF:")
po_file = files.upload()

print("📥 Upload Sales Order (SO) PDF:")
so_file = files.upload()

po_filename = list(po_file.keys())[0]
so_filename = list(so_file.keys())[0]

# 📄 Extract text from PDF
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        return "\n".join([page.extract_text() or "" for page in pdf.pages])

# 🧠 Compare using Gemini
def compare_with_gemini(po_text, so_text):
    prompt = f"""
You are an expert accountant. Compare the following Purchase Order (PO) and Sales Order (SO).

Return a bullet-point summary of:
1. Line items with mismatched amount or quantity.
2. Items present in one but missing in the other.
3. Total value differences.

--- PURCHASE ORDER ---
{po_text}

--- SALES ORDER ---
{so_text}
"""
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    chat = model.start_chat()
    response = chat.send_message(prompt)
    return response.text.strip()

# 📄 Extract text from PDFs
po_text = extract_text_from_pdf(po_filename)
so_text = extract_text_from_pdf(so_filename)

# 🔍 Compare the documents
print("🔍 Comparing PO and SO using Gemini...")
result = compare_with_gemini(po_text, so_text)

# 📊 Display the results
display(Markdown("## ✅ Comparison Result"))
display(Markdown(result))

# 💾 Save result to file
with open("gemini_comparison_result.txt", "w") as f:
    f.write(result)

print("📁 Result saved as gemini_comparison_result.txt")
files.download("gemini_comparison_result.txt")


🔑 Enter your Gemini API Key: ··········
📥 Upload Purchase Order (PO) PDF:


Saving PO-2025-001.pdf to PO-2025-001 (7).pdf
📥 Upload Sales Order (SO) PDF:


Saving SO-2025-022.pdf to SO-2025-022 (7).pdf


🔍 Comparing PO and SO using Gemini...


TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.